In [ ]:
import psycopg2
import pandas as pd
import time
from mbi.utils.Emails import Emails

# Connection parameters, yours will be different
param_dic = {
    "host"      : "argus-db-serverpro.postgres.database.azure.com",
    "database"  : "argus",
    "user"      : "argus@argus-db-serverpro",
    "password"  : "Proyectoeoi2019"
}
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df

conn = connect(param_dic)

column_names = ["id", "timestamp", "publishedat", "title", "summary", "url", "urlToImage", "feedback", "entity", "score"] 

em = Emails(emailsto=['fernando.merino@servexternos.gruposantander.com'], subject="KPIs Digital Mercados Draft", mailid=False)

em.sendmail()

print("Email mandado")
while True:
    df = postgresql_to_dataframe(conn, "select * from argus_dj_articles order by publishedat desc limit 100", column_names)
    print(df.head())
    hora = time.asctime(time.localtime())
    df.to_excel("output.xlsx")
    print(hora)
    time.sleep(10)
